### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
complete_df = complete_df.rename(columns = {"student_name": "Student Name",
                                                    "gender": "Gender",
                                                    "grade": "Grade",
                                                    "school_name": "School Name",
                                                    "reading_score": "Reading Score",
                                                    "math_score": "Math Score",
                                                    "type": "School Type",
                                                    "size":"Number of Students",
                                                    "budget": "Budget"})
complete_df.head()

,Student ID,Student Name,Gender,Grade,School Name,Reading Score,Math Score,School ID,School Type,Number of Students,Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
#Calculate total number of schools
totalschools = len(complete_df["School Name"].unique())

#Calculate total number of students
totalstudents = complete_df["Student ID"].count()
totalstudents

#Calculate total budget
totalbudget = (complete_df["Budget"].unique().sum())
totalbudget

#Calculate average math score
average_math_score = complete_df["Math Score"].mean()
average_math_score 

#Calculate average reading scores
average_reading_score = complete_df["Reading Score"].mean()
average_reading_score

#Calculate overall passing score
overall_average_score = (average_math_score + average_reading_score)/2
overall_average_score

80.43160582078121

In [4]:
#Create df of school data that only includes math scores 70+
passingm = complete_df.loc[complete_df["Math Score"]>=70]

#Calculate dividend by counting number of student IDs in DF
passingmath = passingm["Student ID"].count()

#Calculate percent passing math
pctpassingmath = passingmath/totalstudents*100

In [5]:
#Do the same for reading scores
passingr = complete_df.loc[complete_df["Reading Score"]>=70]
passingreading = passingr["Student ID"].count()
passingreading
pctpassingreading = passingreading/totalstudents*100


In [6]:
#create dataframe
districtsummary = pd.DataFrame(
{"Total Schools": [totalschools],
"Total Students": [totalstudents],
 "Total Budget" : [totalbudget],
"Average Math Score": [average_math_score],
"Average Reading Score": [average_reading_score],
 "Overall Average Score": [overall_average_score],
 "% Passing Math" : [pctpassingmath],
 "% Passing Reading" : [pctpassingreading]
 
})

#Use map to format all columns
districtsummary["Total Students"] = districtsummary["Total Students"].map("{:,}".format)
districtsummary["Total Budget"] = districtsummary["Total Budget"].map("${:,.2f}".format)
districtsummary["Average Math Score"] = districtsummary["Average Math Score"].map("{:.2f}".format)
districtsummary["Average Reading Score"] = districtsummary["Average Reading Score"].map("{:.2f}".format)
districtsummary["Overall Average Score"] = districtsummary["Overall Average Score"].map("{:.2f}".format)
districtsummary["% Passing Math"] = districtsummary["% Passing Math"].map("{:.2f}".format)
districtsummary["% Passing Reading"] = districtsummary["% Passing Reading"].map("{:.2f}".format)
districtsummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428.00",78.99,81.88,80.43,74.98,85.81


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [7]:
#create function to create series of passing grades for math scores and reading scores

def passinggrade(score):
    return score >=70

complete_df["Passing Math"] = complete_df["Math Score"].apply(passinggrade)
complete_df["Passing Reading"] = complete_df["Reading Score"].apply(passinggrade)

#group schools by name and passing math; unstack to create columns
groupedmath = complete_df.groupby(["School Name", "Passing Math"]).size().unstack()

#same for reading
groupedreading = complete_df.groupby(["School Name", "Passing Reading"]).size().unstack()

#merge groupedmath & groupedreading on school name

merged_scores = pd.merge(groupedmath, groupedreading, on="School Name")

#Drop columns with 'False' value (not passing)
merged_passing_scores = merged_scores[["True_x" ,"True_y"]]

#Rename columns
merged_passing_scores = merged_passing_scores.rename(columns = {"True_x": "Students Passing Math", 
                                                               "True_y": "Students Passing Reading"})

merged_passing_scores


,Students Passing Math,Students Passing Reading
School Name,,
Bailey High School,3318,4077
Cabrera High School,1749,1803
Figueroa High School,1946,2381
Ford High School,1871,2172
Griffin High School,1371,1426
Hernandez High School,3094,3748
Holden High School,395,411
Huang High School,1916,2372
Johnson High School,3145,3867


In [8]:
#Average Math & Reading Scores 
#Create a dataframe grouped by schools
groupedaverage = complete_df.groupby(["School Name"])
groupedaverage.count()

,Student ID,Student Name,Gender,Grade,Reading Score,Math Score,School ID,School Type,Number of Students,Budget,Passing Math,Passing Reading
School Name,,,,,,,,,,,,
Bailey High School,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976
Cabrera High School,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
Figueroa High School,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949
Ford High School,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739
Griffin High School,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468
Hernandez High School,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635
Holden High School,427,427,427,427,427,427,427,427,427,427,427,427
Huang High School,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917
Johnson High School,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761


In [9]:
#Calculate Average Math & Reading Scores for each school
school_reading_scores = groupedaverage['Reading Score'].mean()
school_math_scores = groupedaverage['Math Score'].mean()

#Create new DF w/ this info
school_scores = pd.DataFrame({"Average Math Score": school_math_scores,
                                    "Average Reading Score": school_reading_scores})

school_scores


,Average Math Score,Average Reading Score
School Name,,
Bailey High School,77.048432,81.033963
Cabrera High School,83.061895,83.975780
Figueroa High School,76.711767,81.158020
Ford High School,77.102592,80.746258
Griffin High School,83.351499,83.816757
Hernandez High School,77.289752,80.934412
Holden High School,83.803279,83.814988
Huang High School,76.629414,81.182722
Johnson High School,77.072464,80.966394


In [10]:
#merge school_scores and merged_passing_scores

school_score_summary = pd.merge(school_scores, merged_passing_scores, on="School Name")

school_score_summary


,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading
School Name,,,,
Bailey High School,77.048432,81.033963,3318,4077
Cabrera High School,83.061895,83.975780,1749,1803
Figueroa High School,76.711767,81.158020,1946,2381
Ford High School,77.102592,80.746258,1871,2172
Griffin High School,83.351499,83.816757,1371,1426
Hernandez High School,77.289752,80.934412,3094,3748
Holden High School,83.803279,83.814988,395,411
Huang High School,76.629414,81.182722,1916,2372
Johnson High School,77.072464,80.966394,3145,3867


In [11]:
#Create groupby object and group by school name, this time grabbing type, size, and budget

grouped_schools_df = complete_df.groupby(["School Name"]).nth(0)
grouped_schools_df

#remove unnecessary columns
new_grouped_schools_df = grouped_schools_df[["School Type", "Number of Students", "Budget"]]
new_grouped_schools_df



,School Type,Number of Students,Budget
School Name,,,
Bailey High School,District,4976,3124928
Cabrera High School,Charter,1858,1081356
Figueroa High School,District,2949,1884411
Ford High School,District,2739,1763916
Griffin High School,Charter,1468,917500
Hernandez High School,District,4635,3022020
Holden High School,Charter,427,248087
Huang High School,District,2917,1910635
Johnson High School,District,4761,3094650


In [12]:
#merge dataframes
merged_schools_summary = pd.merge(new_grouped_schools_df, school_score_summary, on="School Name")

# Add column and calulate budget per student 
merged_schools_summary["Budget Per Student"]= merged_schools_summary["Budget"]/merged_schools_summary["Number of Students"]


#% Passing Math & Reading
merged_schools_summary["Students Passing Math (%)"] = ((merged_schools_summary["Students Passing Math"]/merged_schools_summary["Number of Students"])*100).astype(float)
merged_schools_summary["Students Passing Reading (%)"] = ((merged_schools_summary["Students Passing Reading"]/merged_schools_summary["Number of Students"])*100).astype(float)

#Overall Passing Rate
merged_schools_summary["Overall Passing Rate (%)"] = (merged_schools_summary["Students Passing Math (%)"] + merged_schools_summary["Students Passing Reading (%)"])/2


#Organize
merged_schools_summary = merged_schools_summary[["School Type", "Number of Students", "Budget", "Budget Per Student", "Average Math Score", "Average Reading Score", "Students Passing Math", "Students Passing Reading", "Students Passing Math (%)", "Students Passing Reading (%)", "Overall Passing Rate (%)"]]

merged_schools_summary.head()

,School Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math (%),Students Passing Reading (%),Overall Passing Rate (%)
School Name,,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,3318,4077,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1749,1803,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,1946,2381,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,1871,2172,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1371,1426,93.392371,97.138965,95.265668


In [13]:
#Use map to format all columns
merged_schools_summary["Number of Students"] = merged_schools_summary["Number of Students"].map("{:,}".format)

merged_schools_summary["Budget"] = merged_schools_summary["Budget"].map("${:,.2f}".format)

merged_schools_summary["Budget Per Student"] = merged_schools_summary["Budget Per Student"].map("${:,.2f}".format) 

merged_schools_summary["Average Math Score"] = pd.to_numeric(merged_schools_summary["Average Math Score"].map("{:.2f}".format))

merged_schools_summary["Average Reading Score"] = pd.to_numeric(merged_schools_summary["Average Reading Score"].map("{:.2f}".format))

merged_passing_scores["Students Passing Math"] = merged_passing_scores["Students Passing Math"].map("{:,}".format)

merged_passing_scores["Students Passing Reading"] = merged_passing_scores["Students Passing Reading"].map("{:,}".format)

merged_schools_summary["Students Passing Math (%)"] = merged_schools_summary["Students Passing Math (%)"].map("{:.2f}".format)

merged_schools_summary["Students Passing Reading (%)"] = merged_schools_summary["Students Passing Reading (%)"].map("{:.2f}".format)

merged_schools_summary["Overall Passing Rate (%)"] = merged_schools_summary["Overall Passing Rate (%)"].map("{:.2f}".format)

merged_schools_summary.head()


,School Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math (%),Students Passing Reading (%),Overall Passing Rate (%)
School Name,,,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,3318,4077,66.68,81.93,74.31
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,1749,1803,94.13,97.04,95.59
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,1946,2381,65.99,80.74,73.36
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,1871,2172,68.31,79.30,73.80
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,1371,1426,93.39,97.14,95.27


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [14]:
top_performing_schools = merged_schools_summary.sort_values("Overall Passing Rate (%)", ascending=False)

top_performing_schools.head()

,School Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math (%),Students Passing Reading (%),Overall Passing Rate (%)
School Name,,,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,1749,1803,94.13,97.04,95.59
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,1525,1591,93.27,97.31,95.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,1371,1426,93.39,97.14,95.27
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,910,923,94.59,95.95,95.27
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,2143,2204,93.87,96.54,95.20


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [15]:
bottom_performing_schools = merged_schools_summary.sort_values("Overall Passing Rate (%)")

bottom_performing_schools.head()

,School Type,Number of Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math (%),Students Passing Reading (%),Overall Passing Rate (%)
School Name,,,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,2654,3208,66.37,80.22,73.29
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,1946,2381,65.99,80.74,73.36
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,1916,2372,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,3145,3867,66.06,81.22,73.64
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,1871,2172,68.31,79.30,73.80


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [28]:
complete_df
#ninth_graders = complete_df.loc[complete_df["Grade"]=="9"]
#pandas split-apply-comine
ninth_graders = complete_df.loc[complete_df["Grade"] == "9th",:]


ninth_graders.head()

,Student ID,Student Name,Gender,Grade,School Name,Reading Score,Math Score,School ID,School Type,Number of Students,Budget,Passing Math,Passing Reading
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,True,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,True,True
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635,True,True
12,12,Brittney Walker,F,9th,Huang High School,64,79,0,District,2917,1910635,True,False
13,13,William Long,M,9th,Huang High School,71,79,0,District,2917,1910635,True,True


## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [18]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.